This notebook provides examples to go along with the [textbook](http://underactuated.csail.mit.edu/stochastic.html).  I recommend having both windows open, side-by-side!

[Click here](http://underactuated.csail.mit.edu/drake.html#notebooks) for instructions on how to run the notebook on Deepnote and/or Google Colab.

In [ ]:
import matplotlib.pyplot as plt
import mpld3
import numpy as np
from pydrake.all import (AddRandomInputs, BasicVector, DiagramBuilder,
                         LeafSystem, PortDataType, PyPlotVisualizer,
                         RandomDistribution, Simulator, StartMeshcat)

from underactuated import running_as_notebook
from underactuated.jupyter import AdvanceToAndVisualize

if running_as_notebook:
    mpld3.enable_notebook()


In [ ]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

# A Bistable System w/ Gaussian Noise

In [ ]:
def dynamics(x, w):
    return x - x**3 + .3 * w

class SimpleStochasticSystem(LeafSystem):

    def __init__(self):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort("noise", 1,
                              RandomDistribution.kGaussian)
        self.DeclareContinuousState(1)
        self.DeclareVectorOutputPort("state", 1, self.CopyStateOut)

    # xdot(t) = x(t) - x^3(t) + w(t)
    def DoCalcTimeDerivatives(self, context, derivatives):
        x = context.get_continuous_state_vector().GetAtIndex(0)
        w = self.EvalVectorInput(context, 0).GetAtIndex(0)
        xdot = dynamics(x, w)
        derivatives.get_mutable_vector().SetAtIndex(0, xdot)

    # y(t) = x(t)
    def CopyStateOut(self, context, output):
        x = context.get_continuous_state_vector().get_value()
        y = output.get_mutable_value()
        y[:] = x


# Note: This is a candidate for moving to a more central location.
class HistogramVisualizer(PyPlotVisualizer):

    def __init__(self,
                 num_samples,
                 bins,
                 xlim,
                 ylim,
                 draw_timestep,
                 figsize=None):
        PyPlotVisualizer.__init__(self,
                                  draw_timestep,
                                  figsize=figsize,
                                  show=False)
        for i in range(0, num_samples):
            self.DeclareVectorInputPort(f"x{i}", 1)
        self.num_samples = num_samples
        self.bins = bins
        self.data = [0]*num_samples
        self.limits = xlim
        self.ax.set_xlim(xlim)
        self.ax.axis("auto")
        self.ax.set_ylim(ylim)
        self.patches = None

    def draw(self, context):
        if (self.patches):
            t = [p.remove() for p in self.patches]
        for i in range(0, self.num_samples):
            self.data[i] = self.EvalVectorInput(context, i).GetAtIndex(0)
        count, bins, self.patches = self.ax.hist(
            self.data, bins=self.bins, range=self.limits, density=True,
            facecolor="b")
        self.ax.set_title("t = " + str(context.get_time()))


def bistable_demo():
    builder = DiagramBuilder()

    num_particles = 1000
    num_bins = 100
    xlim = [-2, 2]
    ylim = [-1, 3.5]
    draw_timestep = .25
    visualizer = builder.AddSystem(
        HistogramVisualizer(num_particles, num_bins, xlim, ylim, draw_timestep))
    x = np.linspace(xlim[0], xlim[1], 100)
    visualizer.ax.plot(x, dynamics(x, 0), "k", linewidth=2)

    for i in range(0, num_particles):
        sys = builder.AddSystem(SimpleStochasticSystem())
        builder.Connect(sys.get_output_port(0), visualizer.get_input_port(i))

    AddRandomInputs(.1, builder)

    diagram = builder.Build()
    simulator = Simulator(diagram)
    simulator.get_mutable_integrator().set_fixed_step_mode(True)
    simulator.get_mutable_integrator().set_maximum_step_size(0.1)

    AdvanceToAndVisualize(simulator, visualizer, 20.0, 1.0)

bistable_demo()

# The Stochastic Van der Pol Oscillator

In [ ]:
class VanDerPolParticles(LeafSystem):

    def __init__(self, num_particles, mu=1.0):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort("noise", num_particles,
                                    RandomDistribution.kGaussian)
        self.DeclareContinuousState(num_particles, num_particles, 0)
        self.DeclareVectorOutputPort("state", BasicVector(2 * num_particles),
                                     self.CopyStateOut)
        self.num_particles = num_particles
        self.mu = mu

    # TODO(russt):  SetRandomState to  [-0.1144;2.0578] + 0.01*randn(...)

    def DoCalcTimeDerivatives(self, context, derivatives):
        # TODO(russt):  Update this to get_position/velocity once those are
        # bound.
        x = context.get_continuous_state_vector().CopyToVector()
        q = x[:self.num_particles]
        qdot = x[self.num_particles:]
        w = self.EvalVectorInput(context, 0).CopyToVector()
        qddot = -self.mu * (q * q - 1) * qdot - q + .5 * w
        derivatives.get_mutable_vector().SetFromVector(
            np.concatenate((qdot, qddot)))

    # y(t) = x(t)
    def CopyStateOut(self, context, output):
        x = context.get_continuous_state_vector().get_value()
        y = output.get_mutable_value()
        y[:] = x


# Note: This is a candidate for moving to a more central location.
class Particle2DVisualizer(PyPlotVisualizer):

    def __init__(self, num_particles, xlim, ylim, draw_timestep):
        PyPlotVisualizer.__init__(self, draw_timestep, show=False)
        self.DeclareVectorInputPort("x", 2 * num_particles)
        self.num_particles = num_particles
        self.ax.set_xlim(xlim)
        self.ax.set_ylim(ylim)
        zero = [0]*num_particles
        self.lines, = self.ax.plot(zero, zero, "b.")

    def draw(self, context):
        xy = self.EvalVectorInput(context, 0).CopyToVector()
        self.lines.set_xdata(xy[:self.num_particles])
        self.lines.set_ydata(xy[self.num_particles:])
        self.ax.set_title("t = " + str(context.get_time()))

def stochastic_van_der_pol():
    builder = DiagramBuilder()

    num_particles = 5000
    xlim = [-5, 5]
    ylim = [-3.25, 3.25]
    draw_timestep = .25
    sys = builder.AddSystem(VanDerPolParticles(num_particles))
    visualizer = builder.AddSystem(
        Particle2DVisualizer(num_particles, xlim, ylim, draw_timestep))
    builder.Connect(sys.get_output_port(0), visualizer.get_input_port(0))
    AddRandomInputs(.1, builder)

    # TODO(russt): Plot nominal limit cycle.

    diagram = builder.Build()
    simulator = Simulator(diagram)
    simulator.set_publish_every_time_step(False)
    simulator.get_mutable_integrator().set_fixed_step_mode(True)
    simulator.get_mutable_integrator().set_maximum_step_size(0.1)

    AdvanceToAndVisualize(simulator, visualizer, 20.0, 1.0)

stochastic_van_der_pol()